In [5]:
import os
import shutil

from keras.models import *
from keras.layers import *
from keras.applications import *
from keras.preprocessing.image import *
from tensorflow.keras.preprocessing import image
import numpy as np

from keras.applications.inception_v3 import InceptionV3, preprocess_input as inception_v3_preprocess_input
from keras.applications.xception import Xception, preprocess_input as xception_preprocess_input
# from keras.applications.resnet50 import ResNet50, preprocess_input as resnet50_preprocess_input


import h5py
import math

In [12]:
# dir = "/ext/Data/distracted_driver_detection/"
dir = "E:\\JupyterWorkSpace\\BBBBBBS"

resnet50_weight_file = "resnet50-imagenet-finetune152.h5"
xception_weight_file = "xception-imagenet-finetune116.h5"
inceptionV3_weight_file = "inceptionV3-imagenet-finetune172.h5"

from tensorflow.keras.models import load_model
# 加载已保存的模型
model_mix = load_model("models/mixed-model.h5")
# 现在你可以使用 model_mix 来进行预测或其他操作


In [7]:
def write_gap_test(tag, MODEL, weight_file, image_size, lambda_func=None, featurewise_std_normalization=True, image_path=None):
    if image_path is None:
        raise ValueError("You must provide a valid image path!")

    # 输入张量
    input_tensor = Input((*image_size, 3))
    x = input_tensor
    if lambda_func:
        x = Lambda(lambda_func)(x)  # 使用lambda函数进行额外的预处理
    
    # 加载基本模型，不包括顶部的全连接层
    base_model = MODEL(input_tensor=x, weights=None, include_top=False)
    # 构建最终的模型，使用全局平均池化
    model = Model(base_model.input, GlobalAveragePooling2D()(base_model.output))
    # 加载权重文件
    model.load_weights("models/" + weight_file, by_name=True)

    print(MODEL.__name__)

    # 定义数据生成器
    gen = ImageDataGenerator(
        featurewise_std_normalization=featurewise_std_normalization,
        samplewise_std_normalization=False,
    )

    # 加载并处理单张图片
    img = image.load_img(image_path, target_size=image_size)
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # 添加批次维度

    # 如果需要进行标准化，可以用gen标准化
    img_array = gen.standardize(img_array)

    # 预测单张图片
    test = model.predict(img_array)
    print("Test prediction shape: {}".format(test.shape))

    # 将预测结果保存到文件
    print("begin create database {}".format(MODEL.__name__))
    with h5py.File(os.path.join("models", tag, "bottleneck_{}_test.h5".format(MODEL.__name__)),'w') as h:
        h.create_dataset("test", data=test)

    print("write_gap_test {} succeeded".format(MODEL.__name__))


In [15]:
print("===== Single =====")
# write_gap_test("Single", ResNet50, resnet50_weight_file, (240, 320))
# write_gap_test("Single", Xception, xception_weight_file, (320, 480), xception_preprocess_input)
# write_gap_test("Single", InceptionV3, inceptionV3_weight_file, (320, 480), inception_v3_preprocess_input)

# image_path = 'C:\\Users\\13904\\Desktop\\tttt.jpg'  # 请确保这条路径是有效的
# image_path = 'C:\\Users\\13904\\Desktop\\tttt.jpg'  # 请确保这条路径是有效的
image_path = 'F:\\JupyterWorkSpace\\BBBBBBS\\train\\c1\\img_115.jpg'  # 请确保这条路径是有效的

write_gap_test(tag="Single", MODEL=ResNet50, weight_file=resnet50_weight_file, image_size=(240, 320), image_path=image_path)
write_gap_test(tag="Single", MODEL=Xception, weight_file=xception_weight_file, image_size=(320, 480), image_path=image_path)


===== Single =====
ResNet50
1/1 [==============================] - 1s 626ms/step
Test prediction shape: (1, 2048)
begin create database ResNet50
write_gap_test ResNet50 succeeded
Xception
1/1 [==============================] - 1s 560ms/step
Test prediction shape: (1, 2048)
begin create database Xception
write_gap_test Xception succeeded


In [16]:
# 不知道为什么,这个需要运行两遍
write_gap_test(tag="Single", MODEL=InceptionV3, weight_file=inceptionV3_weight_file, image_size=(320, 480), image_path=image_path)

InceptionV3
1/1 [==============================] - 1s 855ms/step
Test prediction shape: (1, 2048)
begin create database InceptionV3
write_gap_test InceptionV3 succeeded


In [15]:
# # 最开始的，暂时弃用
# import numpy as np
# from tensorflow.keras.preprocessing import image
# from tensorflow.keras.preprocessing.image import ImageDataGenerator
# import os

# def gen_kaggle_predictions(model, image_paths, model_image_size):
#     # 创建图像数据生成器
#     gen = ImageDataGenerator()

#     for img_path in image_paths:
#         # 加载单张图片并调整大小
#         img = image.load_img(img_path, target_size=(model_image_size, model_image_size))
#         img_array = image.img_to_array(img)
#         img_array = np.expand_dims(img_array, axis=0)  # 扩展维度以适应模型输入

#         # 标准化图像数据（如果需要）
#         img_array = gen.standardize(img_array)

#         # 对单张图片进行预测
#         y_pred = model.predict(img_array, verbose=1)

#         # 获取文件名
#         fname = os.path.basename(img_path)

#         # 打印图片文件名和预测结果
#         print(f"Prediction for {fname}: {y_pred[0]}")

# # 示例调用
# # image_paths = ['path_to_image1.jpg', 'path_to_image2.jpg']  # 传入单张图片的路径列表
# # gen_kaggle_predictions(model, image_paths, model_image_size=224)

# # 打印完成信息
# print("done")


done


In [17]:
def gen_kaggle_predictions(model, X_test, model_image_size):
    # 如果X_test是预先加载的特征数据，直接使用它
    for i, features in enumerate(X_test):
        # 对每一组特征进行预测
        features = np.expand_dims(features, axis=0)  # 扩展维度以适应模型输入
        
        # 对单张特征进行预测
        y_pred = model.predict(features, verbose=1)

        # 打印预测结果
        print(f"Prediction for sample {i}: {y_pred[0]}")
        
        
        print("------y_pred--------")
        print(y_pred)
        # 将 y_pred 扁平化为一维数组
        y_pred = y_pred.flatten()
        # 使用 np.argmax 找到最大概率值的索引
        predicted_class = np.argmax(y_pred)
        # 获取最大预测类别的概率值
        predicted_prob = y_pred[predicted_class]
        # 打印出预测的类别索引及其概率值
        print(f"Predicted class: c{predicted_class}, Probability: {predicted_prob}")


# 示例调用
test_premodels = [
    "bottleneck_ResNet50_test.h5", 
    "bottleneck_Xception_test.h5", 
    "bottleneck_InceptionV3_test.h5",
]
X_test = []
for filename in test_premodels:
    print('------------------' + filename)
    with h5py.File(os.path.join("models", "Single", filename), 'r') as h:
        X_test.append(np.array(h['test']))

X_test = np.concatenate(X_test, axis=1)

# 传入X_test作为特征数据
gen_kaggle_predictions(model_mix, X_test, model_image_size=320)


------------------bottleneck_ResNet50_test.h5
------------------bottleneck_Xception_test.h5
------------------bottleneck_InceptionV3_test.h5
1/1 [==============================] - 0s 16ms/step
Prediction for sample 0: [0.0000000e+00 7.4185172e-29 0.0000000e+00 0.0000000e+00 1.0000000e+00
 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 1.4672487e-29]
------y_pred--------
[[0.0000000e+00 7.4185172e-29 0.0000000e+00 0.0000000e+00 1.0000000e+00
  0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 1.4672487e-29]]
Predicted class: c4, Probability: 1.0


In [14]:
len(X_test[0])

6144

# 汇总

In [ ]:
# dir = "/ext/Data/distracted_driver_detection/"
dir = "E:\\JupyterWorkSpace\\BBBBBBS"

resnet50_weight_file = "resnet50-imagenet-finetune152.h5"
xception_weight_file = "xception-imagenet-finetune116.h5"
inceptionV3_weight_file = "inceptionV3-imagenet-finetune172.h5"

from tensorflow.keras.models import load_model
# 加载已保存的模型
model_mix = load_model("models/mixed-model.h5")
# 现在你可以使用 model_mix 来进行预测或其他操作

def write_gap_test(tag, MODEL, weight_file, image_size, lambda_func=None, featurewise_std_normalization=True, image_path=None):
    if image_path is None:
        raise ValueError("You must provide a valid image path!")

    # 输入张量
    input_tensor = Input((*image_size, 3))
    x = input_tensor
    if lambda_func:
        x = Lambda(lambda_func)(x)  # 使用lambda函数进行额外的预处理
    
    # 加载基本模型，不包括顶部的全连接层
    base_model = MODEL(input_tensor=x, weights=None, include_top=False)
    # 构建最终的模型，使用全局平均池化
    model = Model(base_model.input, GlobalAveragePooling2D()(base_model.output))
    # 加载权重文件
    model.load_weights("models/" + weight_file, by_name=True)

    print(MODEL.__name__)

    # 定义数据生成器
    gen = ImageDataGenerator(
        featurewise_std_normalization=featurewise_std_normalization,
        samplewise_std_normalization=False,
    )

    # 加载并处理单张图片
    img = image.load_img(image_path, target_size=image_size)
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # 添加批次维度

    # 如果需要进行标准化，可以用gen标准化
    img_array = gen.standardize(img_array)

    # 预测单张图片
    test = model.predict(img_array)
    print("Test prediction shape: {}".format(test.shape))

    # 将预测结果保存到文件
    print("begin create database {}".format(MODEL.__name__))
    with h5py.File(os.path.join("models", tag, "bottleneck_{}_test.h5".format(MODEL.__name__)),'w') as h:
        h.create_dataset("test", data=test)

    print("write_gap_test {} succeeded".format(MODEL.__name__))

    
image_path = 'F:\\JupyterWorkSpace\\BBBBBBS\\train\\c1\\img_115.jpg'  # 请确保这条路径是有效的

write_gap_test(tag="Single", MODEL=ResNet50, weight_file=resnet50_weight_file, image_size=(240, 320), image_path=image_path)
write_gap_test(tag="Single", MODEL=Xception, weight_file=xception_weight_file, image_size=(320, 480), image_path=image_path)
# 不知道为什么,这个需要运行两遍
write_gap_test(tag="Single", MODEL=InceptionV3, weight_file=inceptionV3_weight_file, image_size=(320, 480), image_path=image_path)

def gen_kaggle_predictions(model, X_test, model_image_size):
    # 如果X_test是预先加载的特征数据，直接使用它
    for i, features in enumerate(X_test):
        # 对每一组特征进行预测
        features = np.expand_dims(features, axis=0)  # 扩展维度以适应模型输入
        
        # 对单张特征进行预测
        y_pred = model.predict(features, verbose=1)

        # 打印预测结果
        print(f"Prediction for sample {i}: {y_pred[0]}")
        
        
        print("------y_pred--------")
        print(y_pred)
        # 将 y_pred 扁平化为一维数组
        y_pred = y_pred.flatten()
        # 使用 np.argmax 找到最大概率值的索引
        predicted_class = np.argmax(y_pred)
        # 获取最大预测类别的概率值
        predicted_prob = y_pred[predicted_class]
        # 打印出预测的类别索引及其概率值
        print(f"Predicted class: c{predicted_class}, Probability: {predicted_prob}")


# 示例调用
test_premodels = [
    "bottleneck_ResNet50_test.h5", 
    "bottleneck_Xception_test.h5", 
    "bottleneck_InceptionV3_test.h5",
]
X_test = []
for filename in test_premodels:
    print('------------------' + filename)
    with h5py.File(os.path.join("models", "Single", filename), 'r') as h:
        X_test.append(np.array(h['test']))

X_test = np.concatenate(X_test, axis=1)

# 传入X_test作为特征数据
gen_kaggle_predictions(model_mix, X_test, model_image_size=320)


In [18]:
y_pred_resnet50 = model_resnet50.predict(img_array)
y_pred_xception = model_xception.predict(img_array)
y_pred_inception = model_inception.predict(img_array)

print(f"ResNet50 Prediction: {y_pred_resnet50}")
print(f"Xception Prediction: {y_pred_xception}")
print(f"InceptionV3 Prediction: {y_pred_inception}")


NameError: name 'model_resnet50' is not defined